<a href="https://colab.research.google.com/github/tompoles/Portfolio/blob/master/Strato_GS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from termcolor import colored as cl
import openpyxl
import xlrd 

In [ ]:
# Code to get data directly from google sheet - authorization is needed
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
# setup
gc = gspread.authorize(GoogleCredentials.get_application_default())
# read data and put it in a dataframe
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1GAdGLGTwJ5sRcGDXJHbu6mU7_jIljODpDMBTelfg9Fo/edit?ts=601173f2#gid=313717327')
sheets = gsheets.worksheet('import').get_all_values()

In [ ]:
# Transformation of sheet to dataframe
df3 = pd.DataFrame.from_records(sheets)
df3.columns=pd.MultiIndex.from_arrays(df3.iloc[0:5].values) # determination of headers from the sheet
df3=df3.iloc[5:] # determination of headers from the sheet
df3.columns = [' '.join(col).strip() for col in df3.columns.values]

In [ ]:
# Creating new datframe to fully-aplied pandas function
df = df3.copy()
df.to_csv('test.csv') # saving to csv
df = pd.read_csv('test.csv', delimiter=',', header=[0], sep=',\s+', encoding="utf-8", skipinitialspace=True) # reopening for better manipulation
df = df.fillna(0) # filling NA with 0
df = df.drop(columns=['Unnamed: 0']) # dropping unwanted column
df = df.rename(columns={ df.columns[0]: "metric" ,df.columns[1]: "year", df.columns[2]: "quarter", df.columns[3]: "month" }) 

In [ ]:
# Modification of entry table for further possible operation
# df1 = df[df.index > 872 ] # determination of max useful rows -> if there will be changing in core datasets accordingly to adding rows, there is need to do proportional change of df 
# df = df.drop(df1.index, axis=0) # dropping of unnecessarily rows from dataframe

In [ ]:
df['month'] = df.month.replace({'-': 0}, regex=True) # replacing unwanted characters in month string to change the data type
df['month'] = df['month'].astype(int) # Change data type from object to int
df['year'] = df['year'].astype(str)

In [ ]:
#cols = [0,1,2,3,4,6,8,10,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,35,37,39,41,43,45,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,65,67,69,73,75,77,79,81,83,85,87,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,107,111,112,113,114,115,116,117,118,119,120,121,122,124,125,126,127,128,129,130,131,132,133,134,135,137,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,161,162,163,164,166,167,177,182]
cols = [0,1,2,3,4,6,8,10,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,36,38,40,42,44,46,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,66,68,70,74,76,78,80,82,84,86,88,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,108,112,113,114,115,116,117,118,119,120,121,122,123,125,126,127,128,129,130,131,132,133,134,135,136,138,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,162,163,164,165,167,168,183]
df.info()
cols = [df.columns[x] for x in cols]
df = df[cols]
df = df.rename(columns={ df.columns[0]: "metric" ,df.columns[1]: "year", df.columns[2]: "quarter", df.columns[3]: "month" })
df = df[df["month"].isin([1,2,3,4,5,6,7,8,9,10,11,12])]
df = df.set_index(["metric","year","quarter","month"])
df = df.stack().reset_index()
df = df.rename({"level_4":"group", 0:"value"}, axis=1)
df = df.pivot_table(index=['group','year','month'], columns='metric', values=["value"],aggfunc='sum').reset_index()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1169 entries, 0 to 1168
Columns: 192 entries, metric to Unnamed: 192
dtypes: float64(9), int64(1), object(182)
memory usage: 1.7+ MB


In [ ]:
df['year'] = df['year'].map(lambda x: x.rstrip('0').rstrip('.'))
df['group'] = df['group'].str.replace('\d+', '')

In [ ]:
df['group'] = df['group'].replace(' BB Total BB BB/Franchise','B2B')

In [ ]:
# Creating 3 subcolumns to sort data accordint to channel, banner and country
df['channel'] = np.where(df.group.str.contains('B&M'), 'B&M', 'None') 
df['banner'] = np.where(df.group.str.contains('FTB'), 'FTB', 'None')
df['country'] = np.where(df.group.str.contains('CZ'), 'CZ', 'None')

In [ ]:
# Selecting data to channel column according to channel in group string
df['channel'] = np.where(df.group.str.contains('B&M'), 'B&M', df.channel) 
df['channel'] = np.where(df.group.str.contains('E-com'), 'E-com', df.channel) 
df['channel'] = np.where(df.group.str.contains('B2B'), 'B2B', df.channel)
#df['channel'] = np.where(df.group.str.contains('Other'), 'Other', df.channel)
df['channel'] = np.where(df.group.str.contains('Marketplace'), 'Marketplace', df.channel)
df['channel'] = np.where(df.group.str.contains('Teamsales'), 'Teamsales', df.channel)
df['channel'] = np.where(df.group.str.contains('Other.2'), 'Other', df.channel)
# df['channel'] = np.where(df.group.str.contains('xy'), 'xy', df.channel) - possible of adding new acquisitions that have xy in channel

In [ ]:
# Selecting data to banner column according to banner in group string
df['banner'] = np.where(df.group.str.contains('FTB'), 'FTB', df.banner)
df['banner'] = np.where(df.group.str.contains('RUN'), 'RUN', df.banner)
df['banner'] = np.where(df.group.str.contains('B2B'), 'B2B', df.banner)
df['banner'] = np.where(df.group.str.contains('FIT'), 'FIT', df.banner)
df['banner'] = np.where(df.group.str.contains('Other'), 'Other', df.banner)
df['banner'] = np.where(df.group.str.contains('SPORT'), 'SPORT', df.banner)
df['banner'] = np.where(df.group.str.contains('STR'), 'STR', df.banner)
df['banner'] = np.where(df.group.str.contains('Marketplace'), 'Marketplace', df.banner)
df['banner'] = np.where(df.group.str.contains('Other.2'), 'Other', df.banner)
# df['banner'] = np.where(df.group.str.contains('xy'), 'xy', df.banner) - possible of adding new acquisitions that have xy in banner 

In [ ]:
# Selecting data to country column according to country in group string
df['country'] = np.where(df.group.str.contains('CZ'), 'cz', df.country)
df['country'] = np.where(df.group.str.contains('cz'), 'cz', df.country)
df['country'] = np.where(df.group.str.contains('sk'), 'sk', df.country)
df['country'] = np.where(df.group.str.contains('SK'), 'sk', df.country)
df['country'] = np.where(df.group.str.contains('de'), 'de', df.country)
df['country'] = np.where(df.group.str.contains('Unnamed'), 'Unnamed', df.country)
df['country'] = np.where(df.group.str.contains('at'), 'at', df.country)
df['country'] = np.where(df.group.str.contains('be'), 'be', df.country)
df['country'] = np.where(df.group.str.contains('de'), 'de', df.country)
df['country'] = np.where(df.group.str.contains(' com'), 'com', df.country)
df['country'] = np.where(df.group.str.contains('dk'), 'dk', df.country)
df['country'] = np.where(df.group.str.contains(' es'), 'es', df.country)
df['country'] = np.where(df.group.str.contains('fr'), 'fr', df.country)
df['country'] = np.where(df.group.str.contains('hr'), 'hr', df.country)
df['country'] = np.where(df.group.str.contains('hu'), 'hu', df.country)
df['country'] = np.where(df.group.str.contains('it'), 'it', df.country)
df['country'] = np.where(df.group.str.contains('nl'), 'nl', df.country)
df['country'] = np.where(df.group.str.contains('pl'), 'pl', df.country)
df['country'] = np.where(df.group.str.contains('ro'), 'ro', df.country)
df['country'] = np.where(df.group.str.contains('de'), 'de', df.country)
df['country'] = np.where(df.group.str.contains('se'), 'se', df.country)
df['country'] = np.where(df.group.str.contains('uk'), 'uk', df.country)
df['country'] = np.where(df.group.str.contains('11TS CEE'), '11TS CEE', df.country)
df['country'] = np.where(df.group.str.contains('CEE'), 'CEE', df.country)
#df['country'] = np.where(df.group.str.contains('STR'), 'CEE', df.country)
df['country'] = np.where(df.group.str.contains('B2B'), 'B2B', df.country)
df['country'] = np.where(df.group.str.contains('E-com  Other'), 'Other', df.country)
df['country'] = np.where(df.group.str.contains('Other.2'), 'Other', df.country)
df['country'] = np.where(df.group.str.contains('Marketplace'), 'Marketplace', df.country)
df['country'] = np.where(df.group.str.contains('ie'), 'ie', df.country)
df['country'] = np.where(df.group.str.contains('bg'), 'bg', df.country)
df['country'] = np.where(df.group.str.contains('lv'), 'lv', df.country)
df['country'] = np.where(df.group.str.contains('ee'), 'ee', df.country)
df['country'] = np.where(df.group.str.contains('si'), 'si', df.country)
df['country'] = np.where(df.group.str.contains('lt'), 'lt', df.country)
df['country'] = np.where(df.group.str.contains('gr'), 'gr', df.country)
df['country'] = np.where(df.group.str.contains('fi'), 'fi', df.country)
df['country'] = np.where(df.group.str.contains('pt'), 'pt', df.country)
# df['country'] = np.where(df.group.str.contains('xy'), 'xy', df.country) - possible of adding new acquisitions that have xy in country

In [ ]:
# Modification of date columns
df['monthdays'] = pd.to_datetime(df['year'].astype(str)  + \
                                   df['month'].astype(str), format='%Y%m').dt.daysinmonth # definition of number of days in selected month
df['year'] = df['year'].astype(str) # change of data type
df['month'] = df['month'].astype(str) # change of data type
df['date'] = df['year'].astype(str)+'-'+df['month'] # change of data type and concating year and month
df['date'] = pd.to_datetime(df['date'], format='%Y-%m') # change of data type to 'date' type for further modification

In [ ]:
# dropping the index for subsequent manipulation 
df.columns = df.columns.map(''.join)

In [ ]:
# renaming of the selected columns
df = df.rename(columns={df.columns[0]: "jointable", df.columns[3]: "DEL" , df.columns[4]: "Delivery%", df.columns[5]: "EBIT", df.columns[6]: "Fees",
                        df.columns[7]: "Fees%", df.columns[8]: "GM", df.columns[9]: "GM%", df.columns[10]: "Kickbacks",
                        df.columns[11]: "MIR" , df.columns[12]: "MIR%", df.columns[13]: "Mktg systems", df.columns[14]: "Net Contribution",
                        df.columns[15]: "OPEX" , df.columns[16]: "Other OPEX", df.columns[17]: "Other Marketing", df.columns[18]: "Outsourcing",
                        df.columns[19]: "Returns" , df.columns[20]: "Returns%", df.columns[21]: "Revenues", df.columns[22]: "Wages",
                        df.columns[23]: "Warehousing"})

In [ ]:
# Functions to rearrange a columns position in a table 
def movecol(df, cols_to_move=[], ref_col='', place='After'):
    
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [ ]:
# perform the 'movecol' function 
df = movecol(df, 
             cols_to_move=['channel','banner', 'country', 'monthdays', 'date'], 
             ref_col='jointable',
             place='After')

In [ ]:
# creation of daily dates and daily date 
df = df.loc[df.index.repeat(df.monthdays)] # insert multiple rows as is the count of the monthdays
df['day'] = df.groupby(level=0).cumcount() + 1 # sequential nums of days
df['date'] = pd.to_datetime(df[["year", "month", "day"]]) # defined of date from previous defined variables
df['date'] = df['date'].dt.date # removing default time from date coloumn
df = df.drop(columns=['day']) # drop day coloumn

In [ ]:
# removing the whitespace in df strings
df['EBIT'].replace('\s+', '',regex=True,inplace=True)
df['DEL'].replace('\s+', '',regex=True,inplace=True)
df['Fees'].replace('\s+', '',regex=True,inplace=True)
df['GM'].replace('\s+', '',regex=True,inplace=True)
df['Kickbacks'].replace('\s+', '',regex=True,inplace=True)
df['MIR'].replace('\s+', '',regex=True,inplace=True)
df['Mktg systems'].replace('\s+', '',regex=True,inplace=True)
df['Net Contribution'].replace('\s+', '',regex=True,inplace=True)
df['OPEX'].replace('\s+', '',regex=True,inplace=True)
df['Other OPEX'].replace('\s+', '',regex=True,inplace=True)
df['Other Marketing'].replace('\s+', '',regex=True,inplace=True)
df['Outsourcing'].replace('\s+', '',regex=True,inplace=True)
df['Returns'].replace('\s+', '',regex=True,inplace=True)
df['Revenues'].replace('\s+', '',regex=True,inplace=True)
df['Wages'].replace('\s+', '',regex=True,inplace=True)
df['Warehousing'].replace('\s+', '',regex=True,inplace=True)

In [ ]:
# Uprava formy promennych
df['monthdays'] = df['monthdays'].astype(int) # change of data type to int
# df1.replace('\s+', '',regex=True,inplace=True) # removing the whitespace in df strings
df['EBIT'] = df['EBIT'].str.strip('( )').str.strip('%').replace(',', '.', regex=True).astype(float) # getting rid of ( ) in EBIT var and change to float

In [ ]:
df['DEL'] = df['DEL'].replace("-", np.nan)
df['DEL'] = df['DEL'].replace({',': '.'}, regex=True)
df['DEL'] = df['DEL'].str.rstrip('%').astype(float) / 100.0

In [ ]:
# df['Delivery%'] = df['Delivery%'].replace("-", np.nan)
# df['Delivery%'] = df['Delivery%'].replace({',': '.'}, regex=True)
# df['Delivery%'] = df['Delivery%'].str.rstrip('%').astype(float) / 100.0
# df['Fees%'] = df['Fees%'].replace("-", np.nan)
# df['Fees%'] = df['Fees%'].replace({',': '.'}, regex=True)
# df['Fees%'] = df['Fees%'].str.rstrip('%').astype(float) / 100.0
# df['GM%'] = df['GM%'].replace("-", np.nan)
# df['GM%'] = df['GM%'].replace({',': '.'}, regex=True)
# df['GM%'] = df['GM%'].str.rstrip('%').astype(float) / 100.0
# df['MIR%'] = df['MIR%'].replace("-", np.nan)
# df['MIR%'] = df['MIR%'].replace({',': '.'}, regex=True)
# df['MIR%'] = df['MIR%'].str.rstrip('%').astype(float) / 100.0
# df['Returns%'] = df['Returns%'].replace("-", np.nan)
# df['Returns%'] = df['Returns%'].replace({',': '.'}, regex=True)
# df['Returns%'] = df['Returns%'].str.rstrip('%').astype(float) / 100.0
# df['MIR'] = df['MIR'].replace("+", np.nan)
# df['MIR'] = df['MIR'].replace({',': '.'}, regex=True)
# df['MIR'] = df['MIR'].str.rstrip('%').astype(float) / 100.0
# df['Fees'] = df['Fees'].replace("+", np.nan)
# df['Fees'] = df['Fees'].replace({',': '.'}, regex=True)
# df['Fees'] = df['Fees'].str.rstrip('%').astype(float) / 100.0
# df['GM'] = df['GM'].replace("+", np.nan)
# df['GM'] = df['GM'].replace({',': '.'}, regex=True)
# df['GM'] = df['GM'].str.rstrip('%').astype(float) / 100.0
# df['Returns'] = df['Returns'].replace("+", np.nan)
# df['Returns'] = df['Returns'].replace({',': '.'}, regex=True)
# df['Returns'] = df['Returns'].str.rstrip('%').astype(float) / 100.0
# df['Revenues'] = df['Returns'].replace("+", np.nan)
# df['Returns'] = df['Returns'].replace({',': '.'}, regex=True)
# df['Returns'] = df['Returns'].str.rstrip('%').astype(float) / 100.0

In [ ]:
# Change of the data type of calculated variables from object to float
df[['DEL', 'Fees', 'GM', 'Kickbacks', 
     'MIR', 'Mktg systems', 'Net Contribution', 'OPEX', 'Other OPEX', 
     'Other Marketing', 'Outsourcing', 'Returns', 'Revenues', 'Wages', 
     'Warehousing']] = df[['DEL',  'Fees', 'GM',
                            'Kickbacks', 'MIR', 'Mktg systems', 'Net Contribution', 'OPEX', 'Other OPEX', 'Other Marketing', 
                            'Outsourcing', 'Returns', 'Revenues', 'Wages', 'Warehousing']].astype(float)

In [ ]:
# div calculated variables by monthdays count to get data by days
df[['DEL', 'EBIT',  'Fees', 'GM', 'Kickbacks', 
    'MIR', 'Mktg systems', 'Net Contribution', 
    'OPEX', 'Other OPEX', 'Other Marketing', 
    'Outsourcing', 'Returns', 'Revenues', 'Wages', 
    'Warehousing']] = df[['DEL', 'EBIT', 'Fees', 
    'GM', 'Kickbacks', 'MIR', 'Mktg systems', 
    'Net Contribution', 'OPEX', 'Other OPEX', 'Other Marketing', 
    'Outsourcing', 'Returns', 'Revenues', 'Wages', 'Warehousing']].div(df['monthdays'].values,axis=0).round(4)

In [ ]:
df['Join short'] = df['channel'] + '-' + df['banner'] + '-' + df['country']

In [ ]:
# # Code to get data directly from google sheet - authorization is needed
gc2 = gspread.authorize(GoogleCredentials.get_application_default())
# read data and put it in a dataframe
gsheets2 = gc2.open_by_url('https://docs.google.com/spreadsheets/d/1UsHDfi6_vECaUwzWwWUoBvWe7LsqaW_wQhcW6YpGfVk/edit?ts=603667d3#gid=1974251602')
sheets2 = gsheets2.worksheet('AOV').get_all_values()
sheets3 = gsheets2.worksheet('CR').get_all_values()
sheets4 = gsheets2.worksheet('Delivery Ratio').get_all_values()

In [ ]:
df_aov = pd.DataFrame.from_records(sheets2)
df_cr = pd.DataFrame.from_records(sheets3)
df_cr.columns = df_cr.iloc[0]
df_cr = df_cr.drop(df_cr.index[0])
df_del_rat = pd.DataFrame.from_records(sheets4)
df_del_rat.columns = df_del_rat.iloc[0]
df_del_rat = df_del_rat.drop(df_del_rat.index[0])

In [ ]:
df_aov.columns = ['banner', 'AOV']
df_aov = df_aov.apply(lambda x: x.str.replace(',','.'))
df_aov['AOV'] = df_aov['AOV'].astype(float)
df_cr = df_cr.apply(lambda x: x.str.replace(',','.'))
df_cr['CR'] = df_cr['CR'].astype(float)
df_del_rat = df_del_rat.apply(lambda x: x.str.replace(',','.'))
df_del_rat['Delivery ratio real'] = df_del_rat['Delivery ratio real'].str.rstrip('%').astype('float') / 100.0

In [ ]:
# Creating of the AOV column
result = pd.merge(df, df_aov, how="left", on=['banner'])

In [ ]:
# Creating of the CR column
# df_cr['Join'] = df_cr['Join'].replace('-', ' ',regex=True)
df_cr.rename(columns={'Join':'Join short'}, inplace=True)
# result['jointable'] = result['jointable'].replace('-', ' ',regex=True)

In [ ]:
result2 = pd.merge(result, df_cr, how='left', on=['Join short'])

In [ ]:
# Creating of the Delivery ratio real column
df_del_rat.rename(columns={'Join Plan 2021':'Join short'}, inplace=True)
# df_del_rat['jointable'] = df_del_rat['jointable'].replace('-', ' ',regex=True)
result3 = pd.merge(result2, df_del_rat, how="left", on=['Join short'])

In [ ]:
df = result3.copy()

In [ ]:
# assigning data only to a specified channel
df['AOV'] = np.where(df.channel.str.contains('E-com'), df.AOV, 0)

In [ ]:
# Creating 4 subcolumns to calculate new variables
df['Order value'] = df['Revenues']/df['Delivery ratio real']
df['Conversions'] = df['Order value']/df['AOV']
df['Visits'] = df['Conversions']/df['CR']
df['CPV '] = df['MIR']/df['Visits']

In [ ]:
# adjust df from % to float
df['Delivery%'] = df['Delivery%'].replace("-", np.nan)
df['Delivery%'] = df['Delivery%'].replace({',': '.'}, regex=True)
df['Delivery%'] = df['Delivery%'].str.rstrip('%').astype(float) / 100.0
df['Fees%'] = df['Fees%'].replace("-", np.nan)
df['Fees%'] = df['Fees%'].replace({',': '.'}, regex=True)
df['Fees%'] = df['Fees%'].str.rstrip('%').astype(float) / 100.0
df['GM%'] = df['GM%'].replace("-", np.nan)
df['GM%'] = df['GM%'].replace({',': '.'}, regex=True)
df['GM%'] = df['GM%'].str.rstrip('%').astype(float) / 100.0
df['MIR%'] = df['MIR%'].replace("-", np.nan)
df['MIR%'] = df['MIR%'].replace({',': '.'}, regex=True)
df['MIR%'] = df['MIR%'].str.rstrip('%').astype(float) / 100.0
df['Returns%'] = df['Returns%'].replace("-", np.nan)
df['Returns%'] = df['Returns%'].replace({',': '.'}, regex=True)
df['Returns%'] = df['Returns%'].str.rstrip('%').astype(float) / 100.0

In [ ]:
# Final modifications
df_mod = df.copy()
df_mod = df_mod[df.year != 2019]
df_mod.set_index('jointable', inplace=True)
df_mod = df_mod.rename(columns={'Delivery ratio real':'Delivery ratio'})

In [ ]:
df_mod = df_mod[~df_mod['year'].isin(['2019'])]

In [ ]:
#df_mod['channel'] = np.where(df_mod.jointable.str.contains('E com'), 'E-com', df.channel)

In [ ]:
# Data preparation for Azure import
df_mod[['year','month', 'Conversions', 'Visits', 'EBIT', 
    'Fees', 'GM', 'Kickbacks', 'MIR', 'Mktg systems', 'Net Contribution',  
    'DEL', 'Other Marketing',  'Returns', 'Revenues', 'Wages', 'OPEX', 'Other OPEX', 'Warehousing']] = df_mod[['year','month', 'Conversions', 'Visits', 'EBIT', 
    'Fees', 'GM', 'Kickbacks', 'MIR', 'Mktg systems', 'Net Contribution',  
    'DEL', 'Other Marketing', 'Returns', 'Revenues', 'Wages', 'OPEX', 'Other OPEX','Warehousing']].fillna(0).astype(np.int64)

In [ ]:
df_mod['Order value'] = df_mod['Order value'].round(2)

In [ ]:
df_mod.to_excel('test2.xlsx')

In [ ]:
df_mod.info()

<class 'pandas.core.frame.DataFrame'>
Index: 140288 entries,  E-com  FIT pt to  E-com  FIT pl
Data columns (total 36 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   channel           140288 non-null  object 
 1   banner            140288 non-null  object 
 2   country           140288 non-null  object 
 3   monthdays         140288 non-null  int64  
 4   date              140288 non-null  object 
 5   year              140288 non-null  int64  
 6   month             140288 non-null  int64  
 7   DEL               140288 non-null  int64  
 8   Delivery%         88710 non-null   float64
 9   EBIT              140288 non-null  int64  
 10  Fees              140288 non-null  int64  
 11  Fees%             88808 non-null   float64
 12  GM                140288 non-null  int64  
 13  GM%               93580 non-null   float64
 14  Kickbacks         140288 non-null  int64  
 15  MIR               140288 non-null  int64  
 16  MIR%